## Compare fitness function and EA 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Define a function to plot the evolution of the specified metric (mean, variance, max)
def plot_fitness_evolution(EA_variable, fitness_mode, enemy, metric='fitness_values'):
    base_folder = 'Group_96_data_for_plots'
    algorithm_folder = f"{base_folder}/{EA_variable}"
    fitness_folder = f"{algorithm_folder}/fitness_mode_{fitness_mode}"

    # Check if the folder exists
    if not os.path.exists(fitness_folder):
        raise FileNotFoundError(f"Folder {fitness_folder} does not exist.")
    
    # Store data for all run_ids across generations
    generations_data = [[] for _ in range(30)]  # Assuming 30 generations, adjust as needed

    # Loop through run1 to run10, reading data for each generation in each experiment
    for run_id in range(1, 11):  # Assuming 10 experiments
        # Now the run folder includes the enemy number
        run_folder = f"{fitness_folder}/enemy_{enemy}/run{run_id}"  # Updated directory for each enemy
        for generation in range(30):  # Assuming 30 generations
            gen_folder = f"{run_folder}/generation_{generation}_evaluation_enemy_{enemy}"
            metric_file = f"{gen_folder}/{metric}.txt"
            
            if not os.path.exists(metric_file):
                continue  # Skip this generation if the file does not exist
            
            # Read the metric data for each generation
            data = np.loadtxt(metric_file)
            generations_data[generation].append(data)
    
    # Prepare to store the mean, variance, and max for each generation
    means = []
    variances = []
    max_means = []

    # Compute the mean, variance, and mean of the max for each generation
    for generation_data in generations_data:
        # Concatenate all run_id data into one array
        generation_data_flat = np.concatenate(generation_data) if generation_data else np.array([])

        if generation_data_flat.size > 0:
            means.append(np.mean(generation_data_flat))
            variances.append(np.var(generation_data_flat))
            max_means.append(np.mean([np.max(run_data) for run_data in generation_data]))  # Mean of max per generation
        else:
            means.append(np.nan)
            variances.append(np.nan)
            max_means.append(np.nan)

    # Plot the evolution of mean, variance, and max across generations
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Set the left y-axis
    ax1.set_xlabel('Generations', fontsize=14)
    ax1.set_ylabel('Mean / Max value', fontsize=14)
    ax1.plot(range(30), means, marker='o', color='blue', label='Mean value')
    ax1.plot(range(30), max_means, marker='x', color='red', label='Mean of Max value')
    ax1.tick_params(axis='y')
    
    # Set the right y-axis
    ax2 = ax1.twinx()
    ax2.set_ylabel('Variance', fontsize=14)
    ax2.plot(range(30), variances, marker='s', color='green', label='Variance', linestyle='--')
    ax2.tick_params(axis='y')

    # Set the plot title
    metric_label = 'Fitness Value' if metric == 'fitness_values' else 'Individual Gain'
    plt.title(f'{metric_label} Evolution for {EA_variable} (Fitness Mode: {fitness_mode}, Enemy: {enemy})', fontsize=16)

    # Add legends to ensure all curves are displayed in the legend
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.show()

# Example call
enemies = [3, 4, 5]
for enemy in enemies:
    # Plot the evolution of fitness_values
    plot_fitness_evolution(EA_variable="GA", fitness_mode=2, enemy=enemy, metric='fitness_values')
    
    # Plot the evolution of individual_gains
    #plot_fitness_evolution(EA_variable="GA", fitness_mode=1, enemy=enemy, metric='individual_gains')


## test

In [ ]:
import os
import numpy as np
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy" 
# Define a function to compute the mean of max values across runs
def compute_mean_of_max(EA_variable, fitness_mode, enemy, metric='fitness_values'):
    base_folder = 'Group_96_data_for_plots'
    algorithm_folder = f"{base_folder}/{EA_variable}"
    fitness_folder = f"{algorithm_folder}/fitness_mode_{fitness_mode}"

    # Check if the folder exists
    if not os.path.exists(fitness_folder):
        raise FileNotFoundError(f"Folder {fitness_folder} does not exist.")
    
    # List to store the max value from each run
    max_values = []

    # Loop through run1 to run10, reading the max value for each run
    for run_id in range(1, 11):  # Assuming 10 experiments
        run_folder = f"{fitness_folder}/enemy_{enemy}/run{run_id}"
        run_max_values = []

        # Loop through each generation for the current run
        for generation in range(30):  # Assuming 30 generations
            gen_folder = f"{run_folder}/generation_{generation}_evaluation_enemy_{enemy}"
            metric_file = f"{gen_folder}/{metric}.txt"
            
            if not os.path.exists(metric_file):
                continue  # Skip this generation if the file does not exist
            
            # Load the metric data
            data = np.loadtxt(metric_file)
            run_max_values.append(np.max(data))

        if run_max_values:
            max_values.append(np.max(run_max_values))  # Store the max of the maxes for this run
    
    # Compute and return the mean of the max values across all runs
    if max_values:
        return np.mean(max_values)
    else:
        return np.nan

# Example call
enemies = [3,4,5]
for enemy in enemies:
    mean_max_value = compute_mean_of_max(EA_variable="GA", fitness_mode=2, enemy=enemy, metric='fitness_values')
    print(f"Mean of max values for Enemy {enemy}: {mean_max_value}")


## Calculate final best individual *5

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from evoman.environment import Environment
from demo_controller import player_controller
# 设置隐藏层神经元数
n_hidden_neurons = 10

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy" 
    
# 读取神经网络权重
def load_best_individual(EA_variable, fitness_mode, enemy, run):
    base_folder = 'Group_96_data_for_plots'
    file_path = f"{base_folder}/{EA_variable}/fitness_mode_{fitness_mode}/enemy_{enemy}/run{run}/best_individual.txt"
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    
    return np.loadtxt(file_path)

# 模拟神经网络，返回 individual gain
def simulate_best_individual(EA_variable, fitness_mode, enemy, run):
    weights = load_best_individual(EA_variable, fitness_mode, enemy, run)
    experiment_name = "best_individual_simulation"
    
    if not os.path.exists(experiment_name):
        os.makedirs(experiment_name)
    
    env = Environment(
        experiment_name=experiment_name,
        enemies=[enemy],
        playermode="ai",
        player_controller=player_controller(n_hidden_neurons),
        enemymode="static",
        level=2,
        speed="fastest",
        visuals=False,
        randomini="yes"
    )
    
    fitness, player_energy, enemy_energy, time = env.play(pcont=weights)
    individual_gain = player_energy - enemy_energy
    return individual_gain

# 进行模拟并记录结果
def simulate_for_enemies(EA_variable, fitness_mode, runs=10, enemies=[3, 4, 5]):
    results = {}
    for enemy in enemies:
        enemy_results = []
        for run in range(1, runs + 1):
            for i in range(6):
                individual_gain = simulate_best_individual(EA_variable, fitness_mode, enemy, run)
                enemy_results.append(individual_gain)
        results[enemy] = enemy_results
    return results

# 模拟所有组合的结果，并储存到 all_results
def simulate_for_all_conditions(EA_variables, fitness_modes, runs=10, enemies=[3, 4, 5]):
    all_results = {}
    for EA_variable in EA_variables:
        for fitness_mode in fitness_modes:
            result_key = (EA_variable, fitness_mode)
            all_results[result_key] = simulate_for_enemies(EA_variable, fitness_mode, runs, enemies)
    return all_results

# 绘制 boxplot 的函数
def plot_boxplot_results(all_results, EA_variable, fitness_mode, metric='individual_gains'):
    result_key = (EA_variable, fitness_mode)
    results = all_results.get(result_key)

    if results is None:
        raise ValueError(f"No results found for EA_variable={EA_variable} and fitness_mode={fitness_mode}")

    # 将 results 字典转换为绘图数据格式
    data = []
    enemies = []
    for enemy, values in results.items():
        data.extend(values)
        enemies.extend([f"Enemy {enemy}"] * len(values))
    
    # 创建 DataFrame 来准备绘图
    df = pd.DataFrame({'Individual Gain': data, 'Enemy': enemies})
    
    # 绘制 Boxplot
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Enemy', y='Individual Gain', data=df)
    plt.title(f'{metric} for Different Enemies (EA: {EA_variable}, Fitness Mode: {fitness_mode})', fontsize=16)
    plt.xlabel('Enemy')
    plt.ylabel('Individual Gain')
    plt.show()

# 示例调用：模拟所有组合
EA_variables = ["DE"]
fitness_modes = [1, 2]

# 运行模拟并存储所有结果
all_results = simulate_for_all_conditions(EA_variables, fitness_modes, runs=10, enemies=[3, 4, 5])


In [ ]:
# 绘制特定 EA_variable 和 fitness_mode 的 boxplot
a = [1,2]
for fitness_mode in a:
    plot_boxplot_results(all_results, EA_variable="GA", fitness_mode=fitness_mode)